<a href="https://colab.research.google.com/github/alessandronascimento/pyLiBELa/blob/main/Colabs/pyLiBELa_SB2021_cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First things first
### From now on, we will be only using CUDA, a GPU computing API.
### In order to do that, we need to use the GPU runtime from Google Colab that can be selected following the steps in the menu:
### `Runtime > Change Runtime Type > T4 GPU`

In [1]:
#@title Downloading dependencies {display-mode: "form"}

%%capture
! apt-get install python-dev-is-python3 zlib1g-dev libeigen3-dev libgsl-dev libnlopt-cxx-dev
!pip install ipython-autotime
%load_ext autotime
!pip3 install condacolab
import condacolab
condacolab.install()
!mamba install openbabel


time: 45.6 s (started: 2023-09-26 23:02:09 +00:00)


In [1]:
#@title Installing pyLiBELa {display-mode: "form"}

%%capture
! git clone --branch alex-works https://github.com/alessandronascimento/pyLiBELa.git
! mv pyLiBELa/src src
! rm -rf pyLiBELa
! mkdir -p obj
! rm -f Makefile*
! wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/alex-works/Colabs/Makefile
! sed -i 's+-I/usr/include/openbabel3+-I/usr/local/include/openbabel3+g' Makefile
! make -j2

In [3]:
#@title Importing pyLiBELa {display-mode: "form"}
import random

try:
  from pyPARSER import *
  from pyMol2 import *
  from pyWRITER import *
  from pyGrid import *
  from pyCOORD_MC import *
  from pyFindHB import *
  from pyEnergy2 import *
  from pyGaussian import *
  from pyConformer import *
  from pyRAND import *
  from pyMcEntropy import *
  from pySA import *
  from pyOptimizer import *
  from pyMC import *
  from pyFullSearch import *
  from pyDocker import *
  print('pyLiBELa is imported!')
except ImportError:
  print('An ImportError occurred, try running this cell again!')

pyLiBELa is imported!
time: 55.7 ms (started: 2023-09-26 23:21:51 +00:00)


In [4]:
#@title Getting SB2021 data {display-mode: "form"}

%%capture
from google.colab import drive
drive.mount('/content/drive/')
sb_folder = '/content/drive/MyDrive/pyLiBELa/SB/' #@param {type:"string"}

%cd $sb_folder
targets=[]
targets_file = open('list', 'r');
for line in targets_file:
  targets.append(line.strip())
targets_file.close()


time: 2.27 s (started: 2023-09-26 23:21:54 +00:00)


In [5]:
#@title Docking parameters {display-mode: "form"}

import os
import timeit
import numpy as np

Input = PARSER()

# Some default options:
# 1. We are using two processors for Grid calculations;
#

Input.generate_conformers = True;
Input.dock_parallel = True;
Input.parallel_jobs = 2;
Input.write_grids = True
Input.use_grids = True
Input.write_mol2 = True
Input.atom_limit = 150 #@param {type:"number"}
    #atom limit not counting H

# Energy Calculation Parameters:
scoring_function = "0" #@param ["0", "1", "2", "3"]
Input.dielectric_model = "r" #@param ["r", "constant"]
Input.scoring_function = int(scoring_function)
grid_dimension = 30.0 #@param {type:"number"}
Input.grid_spacing = 0.5 #@param {type:"number"}
Input.solvation_alpha = 0.1 #@param {type:"number"}
Input.solvation_beta = -0.005 #@param {type:"number"}

# Optimization parameter:
Input.min_tol = 1E-4;
Input.min_delta = 1E-5;
Input.dock_min_tol = 1E-4;
search_box = 30.0 #@param {type:"number"}
Input.timeout = 30 #@param {type:"number"}
Input.min_timeout = 30 #@param {type:"number"}
Input.overlay_optimizer = "mma" #@param ["mma", "ln_auglag", "subplex", "none"]
Input.energy_optimizer = "mma" #@param ["direct", "isres", "crs", "esch", "stogo", "mma", "simplex", "none"]
if (Input.scoring_function < 3):
  delta = 2.5 #@param {type:"number"}
  Input.deltaij6 = (delta*delta*delta*delta*delta*delta)
  delta_es = 2.5 #@param {type:"number"}
  Input.deltaij_es6 = pow(delta_es, 6);
  Input.deltaij_es3 = (delta_es*delta_es*delta_es)

Input.search_box_x, Input.search_box_y, Input.search_box_z = search_box, search_box, search_box;
Input.x_dim, Input.y_dim, Input.z_dim = grid_dimension, grid_dimension, grid_dimension;
Input.grid_prefix = "McGrid"

time: 1.35 ms (started: 2023-09-26 23:22:03 +00:00)


In [ ]:
#@title Docking {display-mode: "form"}

%load_ext autotime
import os


print('#%10s %10s' % ('TARGET', 'RMSD(Ang)'))

targets_random=random.sample(targets, len(targets));


target_cut = targets[:390]

for target in target_cut:

  folder = sb_folder+target

#  print()
#  print()
  %cd $folder
  subfolder = "self_docking_"+Input.energy_optimizer
  ! mkdir -p $subfolder
  %cd $subfolder

#  !rm -rf *
  if "McLiBELa_dock.mol2.gz" not in os.listdir():

    print()
    print()

    ! cp ../*lig.am1bcc.mol2.gz lig.mol2.gz
    ! cp ../*.rec.clean.mol2.gz rec.mol2.gz


    if "lig2.smi" not in os.listdir():

      print('Generating SMILES.')
      ! obabel -imol2 lig.mol2.gz -osmi -O lig2.smi &> /dev/null #self docking

    else:

      print('Already SMILEd!')



    insmiles = open('lig2.smi', 'r');
    line=insmiles.readline().strip()
    line2 = line.split()
    ligand_smiles = line2[0]
    insmiles.close();

    Lig2 = Mol2()
    ligcheck = Lig2.parse_smiles(Input, ligand_smiles, 'LIG')
  #  if (not ligcheck):
  #    print("#Ops! A problem was identified in ligand %s" % target);

    lig_src =  'lig.mol2.gz'
    rec_src = 'rec.mol2.gz'

    REC = Mol2(Input, rec_src)
    RefLig = Mol2(Input, lig_src)

  #  print("Receptor has %4d atoms." % REC.N)
  #  print('Reference Ligand has %4d atoms' % RefLig.N)
  #  print('Search ligand has %4d atoms' % Lig2.N)

    if (ligcheck):
      Writer = WRITER(Input)
      Coord = COORD_MC()
      HB = FindHB()
      Ene  = Energy2(Input)

      for i in range(len(REC.residue_pointer)-1):
        HB.parse_residue(REC.residue_pointer[i]-1, REC.residue_pointer[i+1]-2, REC.resnames[i], REC, RefLig, 9.0);
      HB.find_ligandHB(lig_src, RefLig);

      #print('The receptor has %5d / %5d HB donors/acceptors around the active site.' % (len(REC.HBdonors), len(REC.HBacceptors)));

      Dock = Docker(Writer)
      center = Coord.compute_com(RefLig)

  #    print()

      start_energy = Ene.compute_ene(REC, RefLig, RefLig.xyz);

      #print('Starting energy: %7.3f kcal/mol' % start_energy);
      #print('Generating grids. This may take a while..')

      #print(Input.overlay_optimizer)
      #print(Input.energy_optimizer)


      start_time = timeit.default_timer()
      Grids = Grid(Input, Writer, REC, center)
      time_grid = timeit.default_timer() - start_time

      time_per_atom = time_grid/REC.N
      #print('Grids computed, and it took %.2f s! That means %.4f s per atom.' %(time_grid,time_per_atom))

      grid_energy = Ene.compute_ene(Grids, RefLig, RefLig.xyz);

      #print('Grid original energy: %7.3f kcal/mol' % grid_energy);
      #print('Grid error: %7.3f %%' % abs(100.*(start_energy-grid_energy)/start_energy));

      #print('Starting docking calculation...')

      start_time = timeit.default_timer()
      Dock.run(REC, Lig2, RefLig, center, Input, Grids, 0)
      time_dock = timeit.default_timer() - start_time

      !rm -rf log.txt

      log = open("log.txt", 'w')

      header = "target\tREC.N\tRefLig.N\tLig2.N\tstart_energy\ttime_grid\tgrid_energy\ttime_dock\n"
      log.write(header)

      data = "%s\t%d\t%d\t%d\t%.3f\t%.3f\t%.3f\t%.3f\n" %(target,REC.N, RefLig.N, Lig2.N, start_energy, time_grid, grid_energy, time_dock)
      log.write(data)

      !echo $data > log.txt
  else:
    print('Target already docked!')



The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
#    TARGET  RMSD(Ang)
/content/drive/MyDrive/pyLiBELa/SB/121P
/content/drive/MyDrive/pyLiBELa/SB/121P/self_docking_mma
Target already docked!
/content/drive/MyDrive/pyLiBELa/SB/181L
/content/drive/MyDrive/pyLiBELa/SB/181L/self_docking_mma
Target already docked!
/content/drive/MyDrive/pyLiBELa/SB/182L
/content/drive/MyDrive/pyLiBELa/SB/182L/self_docking_mma


Generating SMILES.
182L 2603 15 15 -17.806 3.359 -17.668 0.049
/content/drive/MyDrive/pyLiBELa/SB/183L
/content/drive/MyDrive/pyLiBELa/SB/183L/self_docking_mma


Generating SMILES.
183L 2603 17 17 -18.047 3.377 -17.812 0.039
/content/drive/MyDrive/pyLiBELa/SB/184L
/content/drive/MyDrive/pyLiBELa/SB/184L/self_docking_mma


Generating SMILES.
184L 2603 24 24 -20.221 3.360 -20.085 0.617
/content/drive/MyDrive/pyLiBELa/SB/185L
/content/drive/MyDrive/pyLiBELa/SB/185L/self_docking_mma


Generating SMILES.
185L 2603 16 16 -19.014 3.436 -18.767 0.139
/cont